In [1]:
%cd ..

/Users/marc/Documents/DFCI/pnet


In [2]:
from src import pnet_loader
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd

In [4]:
mutations_dict = {"3'Flank": 'Silent',
                  "5'Flank": 'Silent',
                  "5'UTR": 'Silent',
                  "3'UTR": 'Silent',
                  "IGR": 'Silent',
                  "Intron": 'Silent',
                  "lincRNA": 'Silent',
                  "RNA": 'Silent',
                  "Silent": 'Silent',
                  "non_coding_transcript_exon": 'Silent',
                  "upstream_gene": 'Silent',
                  "Splice_Region": 'Silent',
                  "Targeted_Region": 'Silent',
                  'Splice_Site': 'LOF',
                  'Nonsense_Mutation': 'LOF',
                  'Frame_Shift_Del': 'LOF',
                  'Frame_Shift_Ins': 'LOF',
                  'Stop_Codon_Del': 'LOF',
                  'Stop_Codon_Ins': 'LOF',
                  'Nonstop_Mutation': 'LOF',
                  'Start_Codon_Del': 'LOF',
                  'Missense_Mutation': 'Other_nonsynonymous',
                  'In_Frame_Del': 'Other_nonsynonymous',
                  'In_Frame_Ins': 'Other_nonsynonymous',
                  'De_novo_Start_InFrame': 'Other_nonsynonymous',
                  'De_novo_Start_OutOfFrame': 'Other_nonsynonymous',
                  'Start_Codon_Ins': 'Other_nonsynonymous'
                  }

## Load mutation data

In [5]:
maf = pd.read_csv('data/m1000/M1000_CCF.maf', sep='\t')
survival_data = pd.read_csv('data/m1000/M1000_survival_data.txt', sep='\t').set_index('Tumor_Sample_Barcode')
clinical_mapping = pd.read_csv('data/m1000/TCGA_clinical_mapping_and_pathologic_M.txt', sep='\t').set_index('Tumor_Sample_Barcode')

maf = maf[maf['Tumor_Sample_Barcode'].isin(clinical_mapping.index)].copy()
maf['Variant_Classification'] = [mutations_dict[m] for m in maf['Variant_Classification']]
maf = maf[maf['Variant_Classification'] != 'Silent'].copy()

maf_grouped = maf.groupby('Tumor_Sample_Barcode')['Hugo_Symbol'].apply(set).reset_index(name='mut_list').set_index('Tumor_Sample_Barcode')
mutations = pd.DataFrame(columns = maf['Hugo_Symbol'].unique(), index = maf_grouped.index)
mutations.fillna(0, inplace = True)

for i, p in maf_grouped.iterrows():
    for mut in p['mut_list']:
        mutations.loc[i][mut] = 1

mutations = mutations.join(clinical_mapping[['mapping_patient_id']], how='inner').set_index('mapping_patient_id')

In [6]:
mutations.head()

,PRAMEF12,AIM1L,GMEB1,GRIK3,RPE65,CLCA4,COL11A1,NRAS,FLG,KPRP,...,SLC25A5,RAP2C,VGLL1,SPANXN4,RPL10,AAGAB,HIST1H4B,RPS4X,ARMCX1,FATE1
mapping_patient_id,,,,,,,,,,,,,,,,,,,,,
TCGA-3N-A9WB,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
TCGA-3N-A9WC,0,0,0,1,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-3N-A9WD,0,0,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-BF-A1PU,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-BF-A1PV,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


## Load expression data

In [7]:
skcm_exp = pd.read_csv('data/skcm_tcga_pan_can_atlas_2018/data_mrna_seq_v2_rsem_zscores_ref_all_samples.txt',
                       sep='\t').dropna().set_index('Hugo_Symbol').drop(['Entrez_Gene_Id'], axis=1).T
skcm_exp.index = ['-'.join(ind.split('-')[:-1]) for ind in skcm_exp.index]
# skcm_exp.drop(list(skcm_exp.columns[skcm_exp.columns.duplicated()]), axis=1, inplace= True)
# skcm_exp.drop(list(skcm_exp.index[skcm_exp.index.duplicated()]), inplace= True)

In [8]:
skcm_exp.head()

Hugo_Symbol,UBE2Q2P2,HMGB1P1,RNU12-2P,SSX9P,EZHIP,EFCAB8,SRP14P1,TRIM75P,SPATA31B1P,HSPB1P1,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
TCGA-3N-A9WB,0.2287,0.1120,0.3088,-1.1674,-0.7776,-1.3053,-0.6906,-1.4953,-2.8793,-0.7259,...,0.2795,1.5289,-1.4791,-1.0775,-0.5874,0.7428,-0.2728,-2.0876,-2.2139,-0.1152
TCGA-3N-A9WC,0.4992,-0.9303,-0.6854,-1.1674,-0.5525,0.2834,-0.2565,-1.4953,-2.8793,-0.3385,...,-0.0489,-0.5350,0.5245,0.2616,0.3533,-0.7334,-0.1408,-0.3072,-1.2198,0.6850
TCGA-3N-A9WD,-0.3667,-0.2610,-2.0036,-1.1674,-0.6006,0.1596,-0.9029,-1.4953,-2.8793,0.0433,...,-1.2839,-0.5780,-0.6422,-0.2198,0.6876,1.4183,0.2610,-0.4649,-0.5828,-0.0017
TCGA-BF-A1PU,-2.6079,-0.7479,-2.0036,-0.6248,-1.0828,0.4881,1.2143,-1.4953,-2.8793,0.6230,...,-0.0670,-0.0718,-0.1132,-0.1148,0.3637,-1.3046,-1.0278,0.2870,0.7965,-1.7373
TCGA-BF-A1PV,-0.5174,0.3751,-2.0036,-1.1674,-1.0828,-1.3053,0.3070,-1.4953,-2.8793,0.3525,...,0.5413,0.4944,0.6672,0.3759,-0.7236,-1.3046,0.5597,0.2858,-0.6041,1.3473


## Load prediction target

In [9]:
mat_TCGA = pd.read_csv('data/m1000/mat_TCGA.tsv', sep='\t').set_index('Tumor_Sample_Barcode')

mat_TCGA = mat_TCGA.join(clinical_mapping[['mapping_patient_id']], how='inner').set_index('mapping_patient_id')

heterogeneity_y = pd.DataFrame(index=mat_TCGA.index, columns=['dichtomized_heterogeneity'],
                               data=[int(p > mat_TCGA['heterogeneity'].mean()) 
                                     for p in mat_TCGA['heterogeneity'].values])

In [10]:
heterogeneity_y.head()

,dichtomized_heterogeneity
mapping_patient_id,
TCGA-BF-A1PU,0
TCGA-BF-A1PV,1
TCGA-BF-A1PX,0
TCGA-BF-A1PZ,0
TCGA-BF-A1Q0,1


## Generate pnet_loader

In [11]:
genetic_data = {'rna': skcm_exp, 'mut': mutations}

train_dataset, test_dataset = pnet_loader.generate_train_test(genetic_data, heterogeneity_y)

Given 2 Input modalities
Found 421 overlapping indicies
Initializing Train Dataset
Found 15976 overlapping genes
generated input DataFrame of size (295, 31958)
Initializing Test Dataset
Found 15976 overlapping genes
generated input DataFrame of size (126, 31958)


## Generate Reactome Network

In [12]:
from src import ReactomeNetwork

In [13]:
reactome_network = ReactomeNetwork.ReactomeNetwork(train_dataset.get_genes())

Found 15976 overlapping genes


## Generate Pnet

In [14]:
%cd src
import Pnet

/Users/marc/Documents/DFCI/pnet/src


In [25]:
import pytorch_lightning as pl

In [19]:
model = Pnet.PNET_NN(hparams=
                     {'reactome_network':reactome_network, 'nbr_gene_inputs':len(genetic_data), 'dropout':0.2,
                      'additional_dims':0, 'lr':1e-3, 'weight_decay':1e-5}
                    )

In [23]:
train_loader, val_loader = pnet_loader.to_dataloader(train_dataset, test_dataset, 56)

In [37]:
batch_x, additional, batch_y = next(iter(train_loader))

71
189
169
82
72
93
288
237
62
239
203
265
77
244
185
43
282
202
177
290
36
238
260
181
65
161
58
263
160
246
163
135
174
14
49
228
139
30
63
217
272
38
50
165
52
136
131
12
119
91
293
186
32
248
222
252


In [38]:
trainer = pl.Trainer(precision=16, gpus=1, accelerator='mps', max_epochs=100)

trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=val_loader,)

Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name             | Type       | Params
------------------------------------------------
0 | layers           | ModuleList | 163 M 
1 | preds            | ModuleList | 2.5 K 
2 | first_gene_layer | Sequential | 127 K 
3 | drop1            | Dropout    | 0     
4 | attn             | Linear     | 10    
------------------------------------------------
81.7 M    Trainable params
81.7 M    Non-trainable params
163 M     Total params
326.625   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

114
3
91
35
75
112
26
92
15
46
7
78
72
52
63
117
124
13
23
43
32
41
64
111
51
122
47
39
121
89
69
81
109
99
6
49
5
105
60
113
10
87
36
21
123
38
66
53
100
110
31
58
103
98
48
61


/Users/marc/opt/anaconda3/envs/pnet/lib/python3.10/site-packages/torch/amp/autocast_mode.py:202: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


RuntimeError: mat1 and mat2 shapes cannot be multiplied (56x31958 and 31952x2)